In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-23 16:20:54--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-23 16:20:54 (8.16 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [1]:
import minsearch

In [3]:
import json

In [4]:
with open('documents.json', 'rt') as f:
    docs_raw = json.load(f)

In [5]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [8]:
q = "the course has bugun, can I still join?"

In [9]:
index.fit(documents)

In [10]:
boost = {"question": 3.0, "section": 0.5}


results = index.search(
        query = q,
        boost_dict = boost,
        filter_dict = {"course": "data-engineering-zoomcamp"},
        num_results = 5
)

In [11]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineerin

In [12]:
from openai import OpenAI

In [22]:
#client = OpenAI()

In [15]:
result = client.chat.completions.create(
    model = 'gpt-4o',
    messages = [{"role": "user", "content": q}]
)

In [18]:
result.choices[0].message

ChatCompletionMessage(content="It would depend on the specific course and its policies. Many courses have a grace period during which new students can join after the official start date. \n\nHere are a few steps you can take to find out:\n\n1. **Check the Course Website:** Look for any information about late registration or enrollment policies.\n2. **Contact the Instructor:** Send an email or message to the course instructor explaining your situation.\n3. **Reach Out to Administration:** If applicable, contact the administrative office or registrar for the program offering the course.\n4. **Review the Syllabus:** If you have access to the syllabus, it may contain information about late entries.\n\nIf you explain your situation and express your enthusiasm for the course, there's a good chance they might accommodate you.", role='assistant', function_call=None, tool_calls=None)

In [23]:
result.choices[0].message.content

"It would depend on the specific course and its policies. Many courses have a grace period during which new students can join after the official start date. \n\nHere are a few steps you can take to find out:\n\n1. **Check the Course Website:** Look for any information about late registration or enrollment policies.\n2. **Contact the Instructor:** Send an email or message to the course instructor explaining your situation.\n3. **Reach Out to Administration:** If applicable, contact the administrative office or registrar for the program offering the course.\n4. **Review the Syllabus:** If you have access to the syllabus, it may contain information about late entries.\n\nIf you explain your situation and express your enthusiasm for the course, there's a good chance they might accommodate you."

In [24]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FSQ database.
Use only the facts from the CONTEXT when asnwering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: {question}

CONTEXT: {context}
"""
context = ""
for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

prompt = prompt_template.format(question=q, context=context).strip()


In [25]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FSQ database.
Use only the facts from the CONTEXT when asnwering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: the course has bugun, can I still join?

CONTEXT: section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1
Subscribe to course public Google Calendar (it works from Desktop only).
Register

In [26]:
response = client.chat.completions.create(
    model = 'gpt-4o',
    messages = [{"role": "user", "content": prompt}]
)

response.choices[0].message.content

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

## Formulate the pipeline

In [27]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [28]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt


In [29]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [32]:
query = 'Can I use redshift?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [33]:
rag(query)

'Yes, you can use any tool you want for your project, including Redshift. However, please note that if you choose to use a different tool or stack, the course cannot support you with that choice, and you will need to explain your tool choices during the peer review of your capstone project.'